# ⚡ TITAN VIDEO ENGINE (V7.2 - CLOUD EDITION)
### Run TITAN on Google's Free T4 GPU
No Credits | Unlimited Generations | 16GB VRAM

**Instructions:**
1. Click the **Play** button on the cell below.
2. Wait for installation (approx 2-3 mins).
3. Click the **public link** (e.g. `https://xxxx.gradio.live`) at the bottom to open the UI.

In [ ]:
# @title 1. Install Dependencies & Launch TITAN
print("🚀 INSTALLING TITAN ENGINE...")
!pip install -q diffusers transformers accelerate gradio peft opencv-python-headless safetensors huggingface_hub

import torch
from huggingface_hub import hf_hub_download
import time
from diffusers import (
    DiffusionPipeline, 
    DPMSolverMultistepScheduler, 
    StableVideoDiffusionPipeline, 
    StableDiffusionXLPipeline, 
    EulerDiscreteScheduler,
    AnimateDiffPipeline,
    MotionAdapter
)
from diffusers.utils import export_to_video, load_image
from safetensors.torch import load_file
import os
import sys
import cv2
from PIL import Image
import gc
import numpy as np
import gradio as gr

# ==============================================================================
# TITAN ENGINE CORE (EMBEDDED)
# ==============================================================================

class TitanDirector:
    def __init__(self):
        self.device = "cuda" if torch.cuda.is_available() else "cpu"
        self.dtype = torch.float16 if self.device == "cuda" else torch.float32
        self.t2v_pipe = None
        self.i2v_pipe = None
        self.scene_gen_pipe = None
        self.action_pipe = None
        self.quality_mode = "SPEED"
        self.current_character_path = None
        self.last_generated_scene_path = None
        self.VIRAL_PRESETS = {
            "None": "",
            "✨ TikTok Perfect (Standard)": "beautiful young woman, trending on tiktok, perfect face, symmetrical, soft lighting, 8k, ultra realistic, influencer look",
            "🖤 TikTok Artistic (B&W)": "black and white photography, artistic portrait, high contrast, moody, emotive, detailed skin texture, film grain, noir style",
            "🦋 Unique Beauty (Vitiligo)": "beautiful woman with vitiligo, unique skin patterns, detailed skin texture, natural beauty, raw photo, hyperrealistic",
            "🦌 Natural Beauty (Freckles)": "beautiful woman, no makeup, heavy freckles, detailed pores, hyperrealistic skin, raw photography, natural lighting",
            "🦇 Goth/Alt Aesthetic": "gothic style woman, pale skin, dark eyeliner, alternative fashion, moody lighting, edgy, trending on tiktok",
            "💄 Insta Baddie (Glam)": "instagram model, heavy glam makeup, ring light, perfect contour, styled hair, fashion influencer, studio lighting",
            "💪 Fitness/GymTok": "fitness model, athletic physique, gym wear, sweat, workout glow, gym lighting, strong, confident"
        }
        
        print(f"⚡ TITAN DIRECTOR ONLINE. DEVICE: {self.device.upper()}")
        if self.device == "cuda":
            self.vram_gb = torch.cuda.get_device_properties(0).total_memory / 1024**3
            print(f"⚡ VRAM DETECTED: {self.vram_gb:.1f} GB")
        else:
            self.vram_gb = 0

    def get_preset_list(self):
        return list(self.VIRAL_PRESETS.keys())

    def apply_preset(self, prompt, preset_name):
        if not preset_name or preset_name not in self.VIRAL_PRESETS or preset_name == "None":
            return prompt
        base = self.VIRAL_PRESETS[preset_name]
        if prompt:
            return f"{base}, {prompt}"
        return base

    def _unload_all(self):
        if self.t2v_pipe: del self.t2v_pipe
        if self.i2v_pipe: del self.i2v_pipe
        if self.scene_gen_pipe: del self.scene_gen_pipe
        if self.action_pipe: del self.action_pipe
        self.t2v_pipe = None
        self.i2v_pipe = None
        self.scene_gen_pipe = None
        self.action_pipe = None
        gc.collect()
        if self.device == "cuda":
            torch.cuda.empty_cache()
        print("🧹 MEMORY PURGED.")

    def set_character(self, path):
        self.current_character_path = path
        print(f"🔒 CHARACTER LOCKED: {path}")

    def _smart_prompt(self, prompt, is_pony=False):
        prefix = ""
        quality_tags = ", masterpiece, best quality, 8k, cinematic lighting, photorealistic, hdr, sharp focus, high definition, detailed texture, professional photography"
        negative = "blurry, low quality, distorted, bad anatomy, ugly, pixelated, text, watermark, bad hands, missing fingers, extra digit, fewer digits, cropped, jpeg artifacts, signature, username, artist name, deformed, disfigured, extra limbs, missing limbs, floating limbs, disconnected limbs, mutation, mutated, gross proportions, malformed limbs, long neck, duplicate, mutilated, out of frame, body out of frame, clone, duplicate, fused, fused fingers, too many fingers, collapse, glitch"
        return f"{prefix}{prompt}{quality_tags}", negative

    def set_quality(self, mode):
        self.quality_mode = mode.upper()
        print(f"⚡ QUALITY SET TO: {self.quality_mode}")

    def load_scene_builder(self):
        if self.scene_gen_pipe is not None: return
        self._unload_all()
        print("⚡ LOADING SCENE BUILDER (Juggernaut XL Lightning)...")
        try:
            pipe = StableDiffusionXLPipeline.from_pretrained(
                "RunDiffusion/Juggernaut-XL-Lightning",
                torch_dtype=self.dtype
            )
            pipe.load_ip_adapter("h94/IP-Adapter", subfolder="sdxl_models", weight_name="ip-adapter_sdxl.bin")
            pipe.scheduler = DPMSolverMultistepScheduler.from_config(pipe.scheduler.config, use_karras_sigmas=True)
            if self.device == "cuda":
                pipe.enable_model_cpu_offload()
            pipe.vae.enable_tiling()
            self.scene_gen_pipe = pipe
            print("✅ SCENE BUILDER READY.")
        except Exception as e:
            print(f"<!> BUILDER LOAD ERROR: {e}")

    def load_realism_engine(self):
        if self.i2v_pipe is not None: return
        self._unload_all()
        print("⚡ LOADING ANIMATOR (SVD-XT)...")
        try:
            repo_id = "stabilityai/stable-video-diffusion-img2vid-xt"
            pipe = StableVideoDiffusionPipeline.from_pretrained(
                repo_id, 
                torch_dtype=self.dtype, 
                variant="fp16"
            )
            pipe.enable_model_cpu_offload()
            if self.vram_gb < 10:
                pipe.unet.enable_forward_chunking()
            self.i2v_pipe = pipe
            print("✅ ANIMATOR READY (SVD-XT).")
        except Exception as e:
            print(f"<!> SVD LOAD ERROR: {e}")

    def load_action_engine(self):
        if self.action_pipe is not None: return
        self._unload_all()
        print("⚡ LOADING ACTION ENGINE (AnimateDiff Lightning)...")
        try:
            ckpt_path = hf_hub_download("ByteDance/AnimateDiff-Lightning", "animatediff_lightning_4step_diffusers.safetensors")
            adapter = MotionAdapter.from_pretrained("guoyww/animatediff-motion-adapter-v1-5-2", torch_dtype=self.dtype)
            state_dict = load_file(ckpt_path, device="cpu")
            adapter.load_state_dict(state_dict)
            pipe = AnimateDiffPipeline.from_pretrained(
                "emilianJR/epiCRealism",
                motion_adapter=adapter,
                torch_dtype=self.dtype
            )
            pipe.load_ip_adapter("h94/IP-Adapter", subfolder="models", weight_name="ip-adapter_sd15.bin")
            pipe.scheduler = EulerDiscreteScheduler.from_config(
                pipe.scheduler.config, timestep_spacing="trailing", beta_schedule="linear"
            )
            pipe.enable_model_cpu_offload()
            pipe.vae.enable_tiling()
            self.action_pipe = pipe
            print("✅ ACTION ENGINE READY.")
        except Exception as e:
            print(f"<!> ACTION ENGINE ERROR: {e}")

    def create_character(self, prompt):
        print(f"\n📸 GENERATING NEW CHARACTER: '{prompt}'")
        self.load_scene_builder()
        self.scene_gen_pipe.set_ip_adapter_scale(0.0)
        full_prompt, negative = self._smart_prompt(prompt)
        width, height = 832, 1216
        dummy_img = Image.new("RGB", (width, height), (0, 0, 0))
        image = self.scene_gen_pipe(
            full_prompt, 
            negative_prompt=negative, 
            ip_adapter_image=dummy_img,
            height=height,
            width=width,
            num_inference_steps=6, 
            guidance_scale=2.0
        ).images[0]
        filename = f"char_{int(time.time())}.png"
        image.save(filename)
        self.current_character_path = os.path.abspath(filename)
        return self.current_character_path, filename

    def generate_realism(self, motion_val=5, fps=7):
        if not self.current_character_path:
            return None
        self.load_realism_engine()
        image = load_image(self.current_character_path)
        image = image.resize((576, 1024))
        
        # Map 1-10 to SVD bucket
        bucket = int(50 + (motion_val * 15)) # 1->65, 5->125, 10->200
        
        print(f"🎬 GENERATING REALISM CLIP (SVD-XT)...")
        frames = self.i2v_pipe(
            image, 
            decode_chunk_size=2,
            generator=torch.manual_seed(int(time.time())),
            motion_bucket_id=bucket,
            noise_aug_strength=0.1,
            num_frames=25
        ).frames[0]
        filename = f"realism_{os.urandom(4).hex()}.mp4"
        export_to_video(frames, output_video_path=filename, fps=fps)
        return filename

    def generate_action(self, prompt, duration_sec=4, motion_strength=5, upscale=False, progress_callback=None):
        if not self.current_character_path: return None
        self.load_action_engine()
        full_prompt, neg = self._smart_prompt(prompt)
        ref_image = load_image(self.current_character_path)
        self.action_pipe.set_ip_adapter_scale(0.6)
        neg = neg + ", static, motionless, frozen, stationary, picture, photo, glitch, morphing, disjointed"
        
        if self.quality_mode == "CINEMA" or self.quality_mode == "ULTRA":
            self.action_pipe.enable_free_init(method="butterworth", num_iters=3)
        else:
            self.action_pipe.disable_free_init()

        chunk_len, overlap = 16, 4
        effective_len = chunk_len - overlap
        target_frames = duration_sec * 8
        num_chunks = int(target_frames / effective_len) + 1
        all_frames = []

        for i in range(num_chunks):
            msg = f"Rendering Clip {i+1}/{num_chunks}..."
            print(f"   -> {msg}")
            if progress_callback: progress_callback(i, num_chunks, msg)
            steps = 20 if self.quality_mode == "ULTRA" else (12 if self.quality_mode == "CINEMA" else 6)
            output = self.action_pipe(
                prompt=full_prompt,
                ip_adapter_image=ref_image,
                negative_prompt=neg,
                num_frames=16,
                guidance_scale=1.0 + (motion_strength * 0.1),
                width=512, height=768,
                num_inference_steps=steps,
                generator=torch.manual_seed(42 + i*100)
            )
            new_frames = output.frames[0]
            gc.collect()
            if self.device == "cuda": torch.cuda.empty_cache()
            
            if i == 0:
                all_frames.extend(new_frames)
            else:
                prev_segment = all_frames[-overlap:]
                curr_segment = new_frames[:overlap]
                blended_segment = []
                for j in range(overlap):
                    alpha = (j + 1) / (overlap + 1)
                    frame_prev = Image.fromarray(np.array(prev_segment[j]))
                    frame_curr = Image.fromarray(np.array(curr_segment[j]))
                    blended = Image.blend(frame_prev, frame_curr, alpha)
                    blended_segment.append(blended)
                all_frames = all_frames[:-overlap] + blended_segment + new_frames[overlap:]

        if upscale:
            if progress_callback: progress_callback(num_chunks, num_chunks, "Upscaling...")
            upscaled_frames = []
            for frame in all_frames:
                img = np.array(frame)
                img = cv2.cvtColor(img, cv2.COLOR_RGB2BGR)
                img = cv2.resize(img, (1024, 1536), interpolation=cv2.INTER_LANCZOS4)
                kernel = np.array([[-1,-1,-1], [-1,9,-1], [-1,-1,-1]])
                img = cv2.filter2D(img, -1, kernel)
                img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
                upscaled_frames.append(Image.fromarray(img))
            all_frames = upscaled_frames

        filename = f"action_{os.urandom(4).hex()}_long.mp4"
        export_to_video(all_frames, output_video_path=filename, fps=8)
        return filename

# ==============================================================================
# UI LAUNCHER
# ==============================================================================

director = TitanDirector()

with gr.Blocks(theme="ocean", title="TITAN CLOUD") as app:
    gr.Markdown("# ⚡ TITAN CLOUD (Free GPU Edition)")
    
    # STATES
    current_char_img = gr.State(None)
    
    # STEP 1: IDENTITY
    with gr.Group() as step1_group:
        gr.Markdown("## 1️⃣ STEP 1: IDENTITY")
        with gr.Row():
            preset_dd = gr.Dropdown(director.get_preset_list(), label="💎 Viral Style Preset", value="None")
        with gr.Row():
            char_prompt = gr.Textbox(label="Who are they?", placeholder="A cyberpunk girl...", lines=2)
            gen_char_btn = gr.Button("CREATE CHARACTER", variant="primary")
        char_output = gr.Image(label="Generated Character", type="filepath", height=400)
        next_btn_1 = gr.Button("LOCK IDENTITY & NEXT ➡️", variant="secondary")

    # STEP 2: ACTION
    with gr.Group(visible=False) as step2_group:
        gr.Markdown("## 2️⃣ STEP 2: THE ACTION")
        engine_mode = gr.Radio(
            ["Action Director (AnimateDiff)", "Realism Engine (SVD-XT Pro)"], 
            label="Engine", value="Realism Engine (SVD-XT Pro)"
        )
        action_prompt = gr.Textbox(label="Prompt", placeholder="Dancing...", lines=2)
        with gr.Accordion("Settings", open=False):
            quality = gr.Radio(["SPEED", "CINEMA", "ULTRA"], label="Quality", value="CINEMA")
            duration = gr.Slider(5, 30, 10, label="Duration (s)")
            motion = gr.Slider(1, 10, 5, label="Motion")
        
        render_btn = gr.Button("GENERATE VIDEO 🎬", variant="primary")
        video_output = gr.Video(label="Final Video")
        back_btn = gr.Button("⬅️ BACK")

    # EVENTS
    def gen_char(p, preset):
        full_p = director.apply_preset(p, preset)
        path, name = director.create_character(full_p)
        return path
    gen_char_btn.click(gen_char, inputs=[char_prompt, preset_dd], outputs=char_output)
    
    def lock_and_next(img):
        if not img: return {step1_group: gr.update(visible=True)}
        director.set_character(img)
        return {step1_group: gr.update(visible=False), step2_group: gr.update(visible=True)}
    next_btn_1.click(lock_and_next, inputs=char_output, outputs=[step1_group, step2_group])
    
    def go_back():
        return {step1_group: gr.update(visible=True), step2_group: gr.update(visible=False)}
    back_btn.click(go_back, outputs=[step1_group, step2_group])

    def render(eng, p, q, d, m, progress=gr.Progress()):
        director.set_quality(q)
        if "SVD" in eng:
            progress(0, desc="Generating Realism Clip...")
            return director.generate_realism(motion_val=m)
        else:
            def cb(i, t, msg): progress(i/t, desc=msg)
            return director.generate_action(p, d, m, upscale=True, progress_callback=cb)
            
    render_btn.click(render, inputs=[engine_mode, action_prompt, quality, duration, motion], outputs=video_output)

app.queue().launch(share=True, debug=True)